In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile

# Unzip the data folder first

In [25]:
#hc - housing core
#hh - housing hotness
#bls - bureal of labor
#ctr - corporate tax rate
#rm -  rent median
df_hc = pd.read_csv('..\Data\data_zip\data_zip\RDC_Inventory_Core_Metrics_County_History.csv')
df_hh = pd.read_csv('..\Data\data_zip\data_zip\RDC_Inventory_Hotness_Metrics_County_History.csv')
df_bls = pd.read_csv(r'..\Data\data_zip\data_zip\bls-emp_by_cfips.csv')
df_ctr = pd.read_csv(r'..\Data\data_zip\data_zip\Corporate_tax_rate.csv')
df_rm_FY19 = pd.read_excel(r'..\Data\data_zip\data_zip\FY2019_50_County_rev.xlsx')
df_rm_FY20 = pd.read_excel(r'..\Data\data_zip\data_zip\FY2020_50_County_rev.xlsx')
df_rm_FY21 = pd.read_excel(r'..\Data\data_zip\data_zip\FY2021_50_County.xlsx')
df_rm_FY22 = pd.read_excel(r'..\Data\data_zip\data_zip\FY2022_FMR_50_county_rev.xlsx')
df_rm_FY23 = pd.read_excel(r'..\Data\data_zip\data_zip\FY2023_FMR_50_county.xlsx')

C:\Users\jacks\AppData\Local\Temp\ipykernel_10424\3539713529.py:6: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hc = pd.read_csv('..\Data\data_zip\data_zip\RDC_Inventory_Core_Metrics_County_History.csv')
C:\Users\jacks\AppData\Local\Temp\ipykernel_10424\3539713529.py:7: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hh = pd.read_csv('..\Data\data_zip\data_zip\RDC_Inventory_Hotness_Metrics_County_History.csv')


In [13]:
df_bls_d = df_bls[df_bls['first_day_of_month'] >= '2017-01-01']

,series_id,year,period,value,footnote_codes,cfips,first_day_of_month,measure_type
351,LAUCN010010000000003,2017,M01,5.2,NaN,1001,2017-01-01,Unemployment rate
352,LAUCN010010000000003,2017,M02,4.9,NaN,1001,2017-02-01,Unemployment rate
353,LAUCN010010000000003,2017,M03,4.3,NaN,1001,2017-03-01,Unemployment rate
354,LAUCN010010000000003,2017,M04,3.7,NaN,1001,2017-04-01,Unemployment rate
355,LAUCN010010000000003,2017,M05,3.5,NaN,1001,2017-05-01,Unemployment rate
...,...,...,...,...,...,...,...,...
5483039,LAUCN721530000000006,2022,M06,10162.0,NaN,72153,2022-06-01,Labor force
5483040,LAUCN721530000000006,2022,M07,9987.0,NaN,72153,2022-07-01,Labor force
5483041,LAUCN721530000000006,2022,M08,10276.0,P,72153,2022-08-01,Labor force
5483042,LAUCN721530000000006,2022,M09,NaN,N,72153,2022-09-01,Labor force


# Delete data_zip folder to load to Github